# 💬 Chat-Commerce 🛒

## Setup

In [ ]:
# Install packages
%pip install --upgrade python-dotenv langchain chromadb

In [1]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

True

In [2]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Index E-Commerce Data

In [3]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = CSVLoader('./data/home-and-garden.csv', csv_args={
    
})
documents = loader.load()

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(documents, embeddings, persist_directory="./index/")

Running Chroma using direct local API.
DEBUG:Chroma:Logger created
Logger created
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
Successfully imported ClickHouse Connect C data optimizations
INFO:clickhouse_connect.driver.ctypes:Successfully import ClickHouse Connect C/Numpy optimizations
Successfully import ClickHouse Connect C/Numpy optimizations
INFO:clickhouse_connect.json_impl:Using python library for writing JSON byte strings
Using python library for writing JSON byte strings
loaded in 160 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
DEBUG:Chroma:Index saved to ./index//index/index.bin
Index saved to ./index//index/index.bin


## Run Queries

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [17]:
from langchain import VectorDBQA
from langchain.agents import Tool

product_search = VectorDBQA.from_chain_type(
    llm=chat, chain_type="stuff", vectorstore=docsearch
)

tools = [
    Tool(
        name="Product Search",
        func=product_search.run,
        description="useful for when you need to answer find products available in the store",
    ),
]

In [18]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

prefix_default = """
Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. 
Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

prefix = """
You are an AI Shopping Assistant designed to be able to assist the user find the right product in an online store.
You are given the following filtered products in a shop and a conversation. You should try better to understand the user needs and suggest one or more products. 
Provide a conversational answer based on the products provided. If you have more than one product to recommend, show them as bulleted list.
If you can't find the answer in the context below, say politely "Hmm, I'm not sure." Don't try to make up a product which is not.
"""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

ask_ai = initialize_agent(
    tools,
    chat,
    agent="chat-conversational-react-description",
    verbose=True,
    memory=memory,
    prefix=prefix,
    input_variables=["input", "agent_scratchpad"],
)


In [20]:
ask_ai.run("Hey! Can you help me to setup my garden?")



> Entering new AgentExecutor chain...
{
    "action": "Product Search",
    "action_input": "What kind of plants do you want to grow in your garden?"
}DEBUG:Chroma:time to pre process our knn query: 2.86102294921875e-06
time to pre process our knn query: 2.86102294921875e-06
DEBUG:Chroma:time to run knn query: 0.00032401084899902344
time to run knn query: 0.00032401084899902344

Observation: I'm sorry, I don't have that information as it was not provided in the context.
Thought:My apologies for the confusion. Here is a response to your original question:

```json
{
    "action": "Final Answer",
    "action_input": "Of course! What specifically would you like help with in setting up your garden? Are you looking for advice on what plants to grow, how to design your garden, or something else?"
}
```

> Finished chain.


'Of course! What specifically would you like help with in setting up your garden? Are you looking for advice on what plants to grow, how to design your garden, or something else?'

In [22]:
ask_ai.run("Yes. I need pots for my plants")



> Entering new AgentExecutor chain...


ValueError: Could not parse LLM output: Here's a response using the Product Search tool:

```json
{
    "action": "Product Search",
    "action_input": "pots for plants"
}
```

This will allow you to search for pots for your plants and find a variety of options that are available for purchase. You can use this information to choose the pots that best fit your needs and preferences.

In [ ]:
ask_ai.run("Suggest me a pot which is environmentally friendly")

In [19]:
memory.buffer

[]